In [1]:
url = r"sqlite:///C:/Users/blinklet/Documents/Chinook_Sqlite.sqlite"

In [2]:
from sqlalchemy import create_engine

engine = create_engine(url)

In [3]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

Base = automap_base()
Base.prepare(autoload_with=engine)

In [4]:
Album = Base.classes.Album
Artist = Base.classes.Artist
Customer = Base.classes.Customer
Employee = Base.classes.Employee
Genre = Base.classes.Genre
Invoice = Base.classes.Invoice
InvoiceLine = Base.classes.InvoiceLine
MediaType = Base.classes.MediaType
Playlist = Base.classes.Playlist
Track = Base.classes.Track
playlisttrack = Base.metadata.tables['PlaylistTrack']

In [21]:
from sqlalchemy import select

statement = (select(Artist))

In [22]:
print(statement)

SELECT "Artist"."ArtistId", "Artist"."Name" 
FROM "Artist"


In [23]:
import pandas as pd

artists = pd.read_sql_query(sql=statement, con=engine)

print(artists.shape)
print(artists.head(5))

(275, 2)
   ArtistId               Name
0         1              AC/DC
1         2             Accept
2         3          Aerosmith
3         4  Alanis Morissette
4         5    Alice In Chains


In [29]:
statement = (select(Artist).where(Artist.Name=='Alice In Chains'))
print(statement)
print()
from sqlalchemy.dialects import sqlite 
query = statement.compile(dialect=sqlite.dialect(),compile_kwargs={"literal_binds": True})
print(query) # will print the compiled query statement againt the dialect.


SELECT "Artist"."ArtistId", "Artist"."Name" 
FROM "Artist" 
WHERE "Artist"."Name" = :Name_1

SELECT "Artist"."ArtistId", "Artist"."Name" 
FROM "Artist" 
WHERE "Artist"."Name" = 'Alice In Chains'


In [48]:
statement = (
    select(Artist)
    .order_by(Artist.Name)
    .limit(5)
    )

alpha_artists = pd.read_sql_query(sql=statement, con=engine)
print(alpha_artists.shape)
print(alpha_artists.head(5))

(5, 2)
   ArtistId                                               Name
0        43                                       A Cor Do Som
1         1                                              AC/DC
2       230          Aaron Copland & London Symphony Orchestra
3       202                                     Aaron Goldberg
4       214  Academy of St. Martin in the Fields & Sir Nevi...


In [51]:
statement = (
    select(Artist)
    .order_by(func.random())
    .limit(5))

rand_artists = pd.read_sql_query(sql=statement, con=engine)
print(rand_artists.shape)
print(rand_artists.head(5))

(5, 2)
   ArtistId                         Name
0        85                Frank Sinatra
1        16               Caetano Veloso
2       265                 Julian Bream
3       203  Nicolaus Esterhazy Sinfonia
4        93                          JET


In [53]:
statement = (select(Album, Track, Artist)
     .join(Track)
     .join(Artist)
    )
print(statement)

SELECT "Album"."AlbumId", "Album"."Title", "Album"."ArtistId", "Track"."TrackId", "Track"."Name", "Track"."AlbumId" AS "AlbumId_1", "Track"."MediaTypeId", "Track"."GenreId", "Track"."Composer", "Track"."Milliseconds", "Track"."Bytes", "Track"."UnitPrice", "Artist"."ArtistId" AS "ArtistId_1", "Artist"."Name" AS "Name_1" 
FROM "Album" JOIN "Track" ON "Album"."AlbumId" = "Track"."AlbumId" JOIN "Artist" ON "Artist"."ArtistId" = "Album"."ArtistId"


In [56]:

statement = (select(Album.Title.label("Album"),
            Artist.Name.label("Artist"),
            Track.Name.label("Track"),
            Track.Composer, 
            Track.Milliseconds.label("Length"))
     .join(Track)
     .join(Artist)
    )
print(statement)

SELECT "Album"."Title" AS "Album", "Artist"."Name" AS "Artist", "Track"."Name" AS "Track", "Track"."Composer", "Track"."Milliseconds" AS "Length" 
FROM "Album" JOIN "Track" ON "Album"."AlbumId" = "Track"."AlbumId" JOIN "Artist" ON "Artist"."ArtistId" = "Album"."ArtistId"


In [73]:
statement = select(
    Employee.EmployeeId, 
    Employee.FirstName.label("Emp_First_Name"),
    Employee.LastName.label("Emp_Last_Name"), 
    Employee.Title,
    Customer.FirstName.label("Cust_First_Name"),
    Customer.LastName.label("Cust_Last_Name"),
    Customer.Company.label("Cust_Company"),   
).outerjoin(Customer)

dataframe = pd.read_sql_query(sql=statement, con=engine)
print(dataframe.shape)
display(dataframe.head().style.hide(axis="index"))

(64, 7)


EmployeeId,Emp_First_Name,Emp_Last_Name,Title,Cust_First_Name,Cust_Last_Name,Cust_Company
1,Andrew,Adams,General Manager,None,None,None
2,Nancy,Edwards,Sales Manager,None,None,None
3,Jane,Peacock,Sales Support Agent,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.
3,Jane,Peacock,Sales Support Agent,François,Tremblay,None
3,Jane,Peacock,Sales Support Agent,Roberto,Almeida,Riotur


In [80]:
statement = (
    select(
        Employee.EmployeeId, 
        (Employee.FirstName + ' ' + Employee.LastName).label("Employee Name"), 
        Employee.Title,
        func.count(Customer.CustomerId).label("Num_Customers")
    )
    .outerjoin(Customer)
    .group_by(Employee.EmployeeId)
)

dataframe = pd.read_sql_query(sql=statement, con=engine)
print(dataframe.to_string(index=False))

 EmployeeId    Employee Name               Title  Num_Customers
          1     Andrew Adams     General Manager              0
          2    Nancy Edwards       Sales Manager              0
          3     Jane Peacock Sales Support Agent             21
          4    Margaret Park Sales Support Agent             20
          5    Steve Johnson Sales Support Agent             18
          6 Michael Mitchell          IT Manager              0
          7      Robert King            IT Staff              0
          8   Laura Callahan            IT Staff              0


In [83]:
#statement = 'SELECT name FROM sqlite_master WHERE type= "table"'
statement = select(sqlite_master.name)
print(statement)
tables = pd.read_sql_query(statement, url)
print(tables)


NameError: name 'sqlite_master' is not defined

In [85]:
from sqlalchemy import inspect

inspector = inspect(engine)

print(inspector.get_table_names())

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [86]:
from pprint import pprint

pprint(inspector.get_columns("Album"))
print()

[{'autoincrement': 'auto',
  'default': None,
  'name': 'AlbumId',
  'nullable': False,
  'primary_key': 1,
  'type': INTEGER()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'Title',
  'nullable': False,
  'primary_key': 0,
  'type': NVARCHAR(length=160)},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'ArtistId',
  'nullable': False,
  'primary_key': 0,
  'type': INTEGER()}]



In [91]:
print(inspector.get_pk_constraint("PlaylistTrack"))

{'constrained_columns': ['PlaylistId', 'TrackId'], 'name': None}


In [90]:
pprint(inspector.get_foreign_keys("Album"))

[{'constrained_columns': ['ArtistId'],
  'name': None,
  'options': {},
  'referred_columns': ['ArtistId'],
  'referred_schema': None,
  'referred_table': 'Artist'}]


In [102]:
def inspect_table(table_name):

    print(f"Table = {table_name}")
    
    print(f"Columns = ", end="")
    col_names_list = []
    for col in inspector.get_columns(table_name):
        col_names_list.append(col['name'])
    print(*col_names_list, sep=", ")
    
    print(f"Primary Keys = ", end="")
    pk_list = inspector.get_pk_constraint(table_name)
    pk_name_list = pk_list["constrained_columns"]
    print(*pk_name_list, sep=", ")

    fk_list = inspector.get_foreign_keys(table_name)
    if fk_list:
        print(f"Foreign Keys:")
        fk_name_list = []
        fk_reftbl_list = []
        fk_refcol_list = []
        
        for fk in fk_list:
            fk_name_list.append(*fk['constrained_columns'])
            fk_reftbl_list.append(fk['referred_table'])
            fk_refcol_list.append(*fk['referred_columns'])
            
        fk_info = zip(fk_name_list, fk_reftbl_list, fk_refcol_list)
        
        for n, t, c in fk_info:
            print(f"    {n} ---> {t}:{c}")

            
for tbl in inspector.get_table_names():
    inspect_table(tbl)
    print()


Table = Album
Columns = Title, ArtistId, AlbumId
Primary Keys = AlbumId
Foreign Keys:
    ArtistId ---> Artist:ArtistId

Table = Artist
Columns = Name, ArtistId
Primary Keys = ArtistId

Table = Customer
Columns = FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId, CustomerId
Primary Keys = CustomerId
Foreign Keys:
    SupportRepId ---> Employee:EmployeeId

Table = Employee
Columns = LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email, EmployeeId
Primary Keys = EmployeeId
Foreign Keys:
    ReportsTo ---> Employee:EmployeeId

Table = Genre
Columns = Name, GenreId
Primary Keys = GenreId

Table = Invoice
Columns = CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total, InvoiceId
Primary Keys = InvoiceId
Foreign Keys:
    CustomerId ---> Customer:CustomerId

Table = InvoiceLine
Columns = InvoiceId, TrackId, UnitPrice

In [96]:
for table_name in inspector.get_table_names():

    print(f"Table = {table_name}")
    
    print(f"Columns = ", end="")
    col_names_list = []
    for col in inspector.get_columns(table_name):
        col_names_list.append(col['name'])
    print(*col_names_list, sep=", ")
    
    print(f"Primary Keys = ", end="")
    pk_list = inspector.get_pk_constraint(table_name)
    pk_name_list = pk_list["constrained_columns"]
    print(*pk_name_list, sep=", ")

    fk_list = inspector.get_foreign_keys(table_name)
    if fk_list:
        print(f"Foreign Keys:")
        fk_name_list = []
        fk_reftbl_list = []
        fk_refcol_list = []
        
        for fk in fk_list:
            fk_name_list.append(*fk['constrained_columns'])
            fk_reftbl_list.append(fk['referred_table'])
            fk_refcol_list.append(*fk['referred_columns'])
            
        fk_info = zip(fk_name_list, fk_reftbl_list, fk_refcol_list)
        
        for n, t, c in fk_info:
            print(f"    {n} ---> {t}:{c}")

    print()

Table = Album
Columns = Title, ArtistId, AlbumId
Primary Keys = AlbumId
Foreign Keys:
    ArtistId ---> Artist:ArtistId

Table = Artist
Columns = Name, ArtistId
Primary Keys = ArtistId

Table = Customer
Columns = FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId, CustomerId
Primary Keys = CustomerId
Foreign Keys:
    SupportRepId ---> Employee:EmployeeId

Table = Employee
Columns = LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email, EmployeeId
Primary Keys = EmployeeId
Foreign Keys:
    ReportsTo ---> Employee:EmployeeId

Table = Genre
Columns = Name, GenreId
Primary Keys = GenreId

Table = Invoice
Columns = CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total, InvoiceId
Primary Keys = InvoiceId
Foreign Keys:
    CustomerId ---> Customer:CustomerId

Table = InvoiceLine
Columns = InvoiceId, TrackId, UnitPrice

In [99]:
def tbls():
    for table_name in inspector.get_table_names():

        print(f"Table = {table_name}")

        print(f"Columns = ", end="")
        col_names_list = []
        for col in inspector.get_columns(table_name):
            col_names_list.append(col['name'])
        print(*col_names_list, sep=", ")

        print(f"Primary Keys = ", end="")
        pk_list = inspector.get_pk_constraint(table_name)
        pk_name_list = pk_list["constrained_columns"]
        print(*pk_name_list, sep=", ")

        fk_list = inspector.get_foreign_keys(table_name)
        if fk_list:
            print(f"Foreign Keys:")
            fk_name_list = []
            fk_reftbl_list = []
            fk_refcol_list = []

            for fk in fk_list:
                fk_name_list.append(*fk['constrained_columns'])
                fk_reftbl_list.append(fk['referred_table'])
                fk_refcol_list.append(*fk['referred_columns'])

            fk_info = zip(fk_name_list, fk_reftbl_list, fk_refcol_list)

            for n, t, c in fk_info:
                print(f"    {n} ---> {t}:{c}")

        print()

tbls()

Table = Album
Columns = Title, ArtistId, AlbumId
Primary Keys = AlbumId
Foreign Keys:
    ArtistId ---> Artist:ArtistId

Table = Artist
Columns = Name, ArtistId
Primary Keys = ArtistId

Table = Customer
Columns = FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId, CustomerId
Primary Keys = CustomerId
Foreign Keys:
    SupportRepId ---> Employee:EmployeeId

Table = Employee
Columns = LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email, EmployeeId
Primary Keys = EmployeeId
Foreign Keys:
    ReportsTo ---> Employee:EmployeeId

Table = Genre
Columns = Name, GenreId
Primary Keys = GenreId

Table = Invoice
Columns = CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total, InvoiceId
Primary Keys = InvoiceId
Foreign Keys:
    CustomerId ---> Customer:CustomerId

Table = InvoiceLine
Columns = InvoiceId, TrackId, UnitPrice

In [160]:
def inspect_table(table_name):
    tbl_out = f"Table = {table_name}\n"
    #print(tbl_out)
    
    col_out = "Columns = "
    cols_list = inspector.get_columns(table_name)
    for i, c in enumerate(cols_list, 1):
        if i < len(cols_list):
            col_out += (c['name'] + ", ")
        else:
            col_out += c['name'] + "\n"
    
    pk_out = "Primary Keys = "
    pk = inspector.get_pk_constraint(table_name)
    pk_list = pk["constrained_columns"]
    for i, c in enumerate(pk_list):
        if i < len(pk_list) - 1:
            pk_out += (c + ", ")
        else:
            pk_out += (c + "\n")  
    
    fk_out = ""
    fk_list = inspector.get_foreign_keys(table_name)
    if fk_list:
        fk_out = "Foreign Keys:\n"
        fk_name_list = []
        fk_reftbl_list = []
        fk_refcol_list = []
        
        for fk in fk_list:
            fk_name_list.append(*fk['constrained_columns'])
            fk_reftbl_list.append(fk['referred_table'])
            fk_refcol_list.append(*fk['referred_columns'])
            
        fk_info = zip(fk_name_list, fk_reftbl_list, fk_refcol_list)
        
        for n, t, c in fk_info:
            fk_out += f"    {n} ---> {t}:{c}\n"
    
    return("".join([tbl_out, col_out, pk_out, fk_out]))
            
for tbl in inspector.get_table_names():
    print(inspect_table(tbl))


Table = Album
Columns = Title, ArtistId, AlbumId
Primary Keys = AlbumId
Foreign Keys:
    ArtistId ---> Artist:ArtistId

Table = Artist
Columns = Name, ArtistId
Primary Keys = ArtistId

Table = Customer
Columns = FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId, CustomerId
Primary Keys = CustomerId
Foreign Keys:
    SupportRepId ---> Employee:EmployeeId

Table = Employee
Columns = LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email, EmployeeId
Primary Keys = EmployeeId
Foreign Keys:
    ReportsTo ---> Employee:EmployeeId

Table = Genre
Columns = Name, GenreId
Primary Keys = GenreId

Table = Invoice
Columns = CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total, InvoiceId
Primary Keys = InvoiceId
Foreign Keys:
    CustomerId ---> Customer:CustomerId

Table = InvoiceLine
Columns = InvoiceId, TrackId, UnitPrice

In [162]:
print(inspect_table("Album"))

Table = Album
Columns = Title, ArtistId, AlbumId
Primary Keys = AlbumId
Foreign Keys:
    ArtistId ---> Artist:ArtistId



In [174]:
print(f"Album table relationships")
print()
thing_relations = inspect(Album).relationships.items()
pprint(thing_relations)
# for relationship in inspect(Album).relationships:
#     print(dir(relationship))
#     print(f"Relationship: {relationship}")
#     print(f"Direction:    {relationship.direction}")
#     print(f"Joined Table: {relationship.target}")
#     print(f"strategy: {relationship.strategy}")
#     print(f"remote_side: {relationship.remote_side}")
#     print(relationship.items())
    
#     print()

Album table relationships

[('artist', <RelationshipProperty at 0x2223a325f50; artist>),
 ('track_collection',
  <RelationshipProperty at 0x2223a3afa50; track_collection>)]
